In [29]:
## Imports
import math

In [30]:
## Pay-to-exit research questions
## RQ1 For an adversary with \alpha stake, how much does it need to pay to
## exiting validators to achieve \alpha'\leq0.33 stake in the system? In this scenario the adversary would 
## reimburse all expected rewards to the validator for the next 1-2-3 (TBD) years (discounted at present value).
## RQ2 The same as in RQ1 but the adversary only reimburses the opportunity cost, i.e.,
## say the difference between validator rewards - Uniswap LP rewards (assuming Uniswap LP is the 2nd best option for validators).

In [31]:
## Best resource on the protocol: https://eth2book.info/latest/book.pdf

In [32]:
## Pay-to-fork research questions
## RQ1 how much the adversary needs to pay for buying X attestations?
## RQ2 Same as RQ1 but the adversary also needs to cover attestation equivocation penalties for these X bought attestations.

In [33]:
## Some constants that we deem given (maybe periodically update)
stakedETHamount = 34385053 ## (2025.04.01) taken from here https://dune.com/hildobby/eth2-staking 
noOfValidators = 1074533 ## (2025.04.01) taken from here https://dune.com/hildobby/eth2-staking
noEpochsPerYear = 82181.25
maxIssuancePerYear = 940.8659*math.sqrt(noOfValidators)
APR = 2940.21/math.sqrt(noOfValidators)
proposerReward = 0
attestationReward = 0
W_s = 14 # weights page 100
W_t = 26
W_h = 14
W_y = 3 ## sync reward weight
W_p = 12.5 ## proposer weight
W_Sigma = 64 ## defined at page 109
T = stakedETHamount ## defined at page 102
n = 32 ## number of increments (the validator's balance is assumed to be 32 ETH uniformly (for the sake of simplicity))
base_reward_factor = 64 ## https://ethereum.stackexchange.com/questions/153722/base-reward-formula-enquiry
b = (1000000000*base_reward_factor)/math.sqrt(32000000000*noOfValidators) ## base reward per increment
## Attestation penalties page 109 in eth2book
attestation_missing_penalty = 0.625*T*b ## page 107
wrong_source_attestation = (-W_s-W_t)/W_Sigma
MIN_SLASHING_PENALTY_QUOTIENT_BELLATRIX = 1/32 ##defined at page 116
## correlation penalty page 117
## other penalties

In [34]:
## Relevant Pectra-EIPs for this work
## EIP-7002
## EIP-7549 Move the committee index out of the signature: more efficient attestation/signature aggregation 

In [35]:
print(maxIssuancePerYear,"ETH issued per year")
print(APR,"% APR")

975298.6148563978 ETH issued per year
2.8364064971488903 % APR


In [36]:
print(base_reward_factor)

64
